# Multibyte Addition

- Multibyte arithmetic is done on values stored in memory. Each byte is moved into the accumulator as it is needed.
<br><br>
- In multibyte arithmetic, we usually don't work with values that have been converted to pure binary form. We instead work with a data representation system called binary coded decimal (BCD). This is also referred to as packed decimal.

<img src="images/bcd-hex.png">

- We see that in the hex value of equivalent BCD is equal to the decimal value.
<br><br>
- In BCD a byte is split into halves (called nibbles) and both nibbles represent a number between 0 to 9. Values A to F are forbidden. E.g:- 83 when converted to BCD has 8 in the upper nibble and 3 in the lower one.
<br><br>
- To prevent the resulting BCD from being invalid, we use an instruction called SED (SEt Decimal mode). This has no operands, and instructs the processor to use BCD arithmetic until further notice.

## Questions

1) Write a set of instructions that will move the least significant byte of ADDEND1 into the accumulator, then add the least significant byte of ADDEND2 to it using BCD arithmetic. Put the result in the LSByte of ADEND2:-

```assembly
SED
LDX  #1
LDA  ADEND1,X
CLC
ADC  ADEND2,X
STA  ADEND2,X
```

2) Write a set of instruction that will move the ADEND1 byte into the accumulator, add the ADEND2 byte to it, plus the value in the carry flag, store it in the correct byte of ADEND2 (this adds 2 bytes):-

```assembly
ADDER    EQU  *
         SED
         LDX  #1
         CLC
         LDA  ADEND1,X
         ADC  ADEND2,X
         STA  ADEND2,X
         DEX
         LDA  ADEND1,X
         ADC  ADEND2,X
         STA  ADEND2,X
```

3) Adapt the routine in question 2 so that it adds two five-byte numbers and stores the sum:-

```assembly
ADDER    EQU  *
         SED
         LDX  #4
         CLC
ADDBYT   EQU  *
         LDA  ADEND1,X
         ADC  ADEND2,X
         STA  ADEND2,X
         DEX
         BPL  ADDBYT
```

# BCD Conversion

## Questions

1) Read one digit from the terminal and convert it from ASCII to binary. All you have to do is turn off all the bits in the MSN (MS Nibble), leave the result in the accumulator:-

```assembly
JSR  INPUT
JSR  OUPUT
AND  #%00001111
```

2) In BCD it's very important that only valid decimal digits are used. Convert the routine you coded in the previous question into a complete subroutine that gets one valid digit:-
<br><br>
    - Read and echo one character.
<br><br>
    - Validate range '0' to '9' (make sure the character is in the range).
<br><br>
    - If the character is out of range, write an error message and try again.
<br><br>
    - When a valid digit is obtained, convert it to binary and return control. Leave the new value in A.
    
```assembly
GETDIG    EQU  *
          STX  HOLDX
AGAIN     EQU  *
          JSR  INPUT
          JSR  OUTPUT
          CMP  #'0'
          BMI  ERROR
          CMP  #':'      ; COMPARE TO CHARACTER AFTER '9'
          BPL  ERROR
          AND  #%00001111
          LDX  HOLDX
          RTS
ERROR     EQU  *
          LDX  #0
MSGOUT    EQU  *
          LDA  ERRMSG,X
          JSR  OUTPUT
          INX
          CPX  #26
          BNE  MSGOUT
          JMP  AGAIN
HOLDX     DS   1
ERRMSG    ASC  'INVALID DIGIT -- TRY AGAIN'
```

- To extend this for two digits we can take the following steps:-
<br><br>
    - Rotate the first digit into the upper nibble
<br><br>
    - Add in the second digit
- Code a routine that will get two valid decimal digits (by calling GETDIG) and create once BCD byte. Store the byte in ADEND1:-

```assembly
GETNUM    EQU  *
          JSR  GETDIG  ; GET MSBYTE
          ASL  A
          ASL  A
          ASL  A
          ASL  A
          STA  MSB
          JSR  GETDIG  ; GET LSBYTE
          CLC
          ADC  MSB
          STA  ADDEND1
MSB       DS   1
```

3) Complete the routine so that it gets and stores all of ADEND1 (ten digits = five bytes) and ADEND2 (also five bytes). For the user's benefit, start a new line on the display screen after each 10-digit number. You'll want to use a 'nested loop'. Have an outer loop that executes the inner loop twice for the two numbers:-

```assembly
CR        EQU  $0D
LF        EQU  $0A
GETADS    EQU  *
          LDX  #0       ; INITIALIZE INDEX
          LDA  #2       ; CONTROL OUTER LOOP FOR 2 NUMBERS
          STA  COUNT2
GETAD     EQU  *
          LDA  #5       ; CONTROL INNER LOOP FOR 5 BYTES PER NUMBER
          STA  COUNT5
GETNUM    EQU  *
          JSR  GETDIG
          ASL  A
          ASL  A
          ASL  A
          ASL  A
          STA  MSB
          JSR  GETDIG
          CLC
          ADC  MSB
          STA  ADEND1,X  ; IF X > 4, WE'RE IN ADEND2
          INX
          DEC  COUNT5    ; COUNT INNER LOOP
          BNE  GETNUM
          LDA  #CR
          JSR  OUTPUT
          LDA  #LF       
          JSR  OUTPUT
          DEC  COUNT2
          BNE  GETAD     ; GET THE SECOND ADDEND
COUNT2    DS   1
COUNT5    DS   1
ADEND1    DS   5
ADEND2    DS   5
MSB       DS   1
```

4) Code a routine that will:-
<br><br>
    - Get one byte from ADEND2.
<br><br>
    - Split the two nibbles.
<br><br>
    - Convert to ASCII.
<br><br>
    - Write both digits.
<br><br>
    - Repeats (1) to (4) until the entire sum is written out.
    
```assembly
MASCII    EQU  %00110000
WRITIT    EQU  *
          LDX  #0
CONVRT    EQU  *
          LDA  ADEND2,X       ; GET BYTE
          LSR  A
          LSR  A
          LSR  A
          LSR  A
          ORA  #MASCII        ; CONVERT TO ASCII
          JSR  OUTPUT
          LDA  ADEND2,X       ; GET IT AGAIN
          AND  #%00001111     ; USE LOWER NIBBLE
          ORA  #MASCII        ; CONVERT TO ASCII
          JSR  OUTPUT
          INX
          CPX  #5
          BNE  CONVRT
```

- The following program shows the entire program to read and add two ten-digit numbers. This program has some weak points:-
<br><br>
    - The user must type all 10 digits including leading zeros.
<br><br>
    - There aren't any steps to prevent overflow of the sum.
<br><br>
    - The errror message routine dumps the message in the middle of the user's number.
    
```assembly
MASCII    EQU  %00110000
CR        EQU  $0D
LD        EQU  $0A
GETADS    EQU  *
          LDX  #0  ; INITIALIZE INDEX
          LDA  #2  ; CONTROL OUTER LOOP
GETAD     EQU  *
          LDA  #5  ; CONTROL INNER LOOP
          STA  COUNT5
GETNUM    EQU  *
          JSR  GETDIG
          ASL  A
          ASL  A
          ASL  A
          ASL  A
          STA  MSB
          JSR  GETDIG   ; GET LSB
          CLC
          ADC  MSB
          STA  ADEND1,X    ; IF X > 4 WE'RE IN ADEND2
          INX
          DEC  COUNT5
          BNE  GETNUM
          LDA  #CR
          JSR  OUTPUT
          LDA  #LF
          JSR  OUTPUT
          DEC  COUNT2
          BNE  GETAD
ADDER     EQU  *
          SED
          LDX  #4
          CLC
ADDBYT    EQU  *
          LDA  ADEND1,X
          ADC  ADEND2,X
          STA  ADEND2,X
          DEX
          BPL  ADDBYT
WRITIT    EQU  *
          LDX  #0
CONVRT    EQU  *
          LDA  ADEND2,X
          AND  #%111100000 ; USE UPPER NIBBLE
          LSR  A
          LSR  A
          LSR  A
          LSR  A
          ORA  #MASCII
          JSR  OUTPUT
          LDA  ADEND2,X
          AND  #%00001111 ; USE LOWER NIBBLE
          ORA  #MASCII
          JSR  OUTPUT
          INX
          CPX  #5
          BNE  CONVRT
DONE      JMP  DONE
GETDIG    EQU  *
          STX  HOLDX
AGAIN     EQU  *
          JSR  INPUT
          JSR  OUTPUT
          CMP  #'0'
          BMI  ERROR
          CMP  #':'
          BPL  ERROR
          AND  #%00001111
          LDX  HOLDX
          RTS
ERROR     EQU  *
          LDX  #0
MSGOUT    EQU  *
          LDA  ERRMSG,X
          JSR  OUTPUT
          INX
          CPX  #26
          BNE  MSGOUT
          JMP  AGAIN
COUNT2    DS   1
COUNT5    DS   1
ADEND1    DS   5
ADEND2    DS   5
MSB       DS   1
HOLDX     DS   1
ERRMSG    ASC  'INVALID DIGIT -- TRY AGAIN'
```